## Script para obtener el Tipo de Cambio de Banxico (Obligaciones)
## El primer y ultimo día del Mes

In [1]:
# Importing Modules
######################
import datetime as dt
import requests
import pandas as pd
import os 

In [2]:
# Fechas para todo el proceso
start_date = "2021-01-01"
end_date = dt.date.today()

In [3]:
# Fecha actual
today = dt.date.today()
print("Today is = "+str(today))

# Token de Consulta Banxico
token = os.environ.get("token_banxico")

# Clave de Descarga Banxico
obligaciones = "SF60653" # Para Solventar Obligaciones
fix = "SF63528" # Fecha de Determinacion (FIX)

# Threshold de porcentaje de cambio de FX para comprar SHV
threshold = 0.00 # --> Expresar porcentaje como decimal

Today is = 2022-01-11


In [4]:
# Descargando datos de Banxico 
#######################################

# Funcion de Descarga
def descarga_bmx_serie(serie, fechainicio, fechafin, token):
    # Al site de banxico se le pegan los datos de consulta
    url = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/"+serie+"/datos/"+fechainicio+"/"+fechafin
    print(url)
    
    # Se le tienen que pasar Headers
    # Se pasa el token de banxico en un diccionario.
    # Se pasa como un Request con metodo Get
    # Se le solicita el codigo de respuesta al servidor. 
    headers = {"Bmx-Token":token}
    response = requests.get(url,headers=headers)
    status = response.status_code
    
    # Si el estatus esta Ok armar el dataframe
    # Si el estatus esta mal imprimir el Error en la consulta.
    if status == 200:
        # Si el codigo es correcto pasa la respuesta a formato Json
        raw_data = response.json()
        
        # Pasamos las llaves en el Json para crear la serie de datos. 
        data = raw_data["bmx"]["series"][0]["datos"]
        
        # Creamos con la serie un dataframe
        # Volvemos los datos floats en vez de strings
        # Volvemos las fechas a formato fecha
        # Volvemos la fecha la columna indice (Deshabilidado)
        # Regresa el Dataframe
        # Cambia el nombre de la columna "dato"  por tipo de cambio
        global df
        df = pd.DataFrame(data)
        df["dato"] = df["dato"].apply(lambda x: float(x))
        df["fecha"] = pd.to_datetime(df["fecha"], format = "%d/%m/%Y")
        #df.set_index("fecha", inplace = True)
        df.columns = ['Fecha', 'Tipo de Cambio']
        return(df)
    else:
         print(status)

# Ejecutando la Solicitud de Descarga
dolares_bmx = descarga_bmx_serie(obligaciones , str(start_date) , str(end_date), token)

#Mostrar el Data Frame en  Jupiter Notebook
df.head()

https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF60653/datos/2021-01-01/2022-01-11


,Fecha,Tipo de Cambio
0,2021-01-01,19.9352
1,2021-01-02,19.9352
2,2021-01-03,19.9352
3,2021-01-04,19.9352
4,2021-01-05,19.9087


In [5]:
# Volver la fecha el Indice

# Make the Date Column the Index
datetime_index = pd.DatetimeIndex(df['Fecha'].values)

# Creating a new dataframe with the new index and adding the Date column name
df2 = df.set_index(datetime_index).rename_axis('Fecha', axis=1)

# Dropping the extra Date Column
df2.drop('Fecha',axis=1,inplace=True)

df2.tail()

Fecha,Tipo de Cambio
2022-01-07,20.3977
2022-01-08,20.4588
2022-01-09,20.4588
2022-01-10,20.4588
2022-01-11,20.4237


In [6]:
# Listas con Inicio y fin de mes para el rango
SOM_dates = pd.date_range(start=start_date, end=end_date, freq='MS')
EOM_dates = pd.date_range(start=start_date, end=end_date, freq='M')

print(SOM_dates)
print(EOM_dates)

DatetimeIndex(['2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01',
               '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01',
               '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
               '2022-01-01'],
              dtype='datetime64[ns]', freq='MS')
DatetimeIndex(['2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30',
               '2021-05-31', '2021-06-30', '2021-07-31', '2021-08-31',
               '2021-09-30', '2021-10-31', '2021-11-30', '2021-12-31'],
              dtype='datetime64[ns]', freq='M')


In [7]:
df3 = df2.loc[SOM_dates]
df4 = df2.loc[EOM_dates]

df3.head()

Fecha,Tipo de Cambio
2021-01-01,19.9352
2021-02-01,20.2697
2021-03-01,20.8523
2021-04-01,20.6025
2021-05-01,20.0978


In [8]:
# Juntando los Dataframes
df_final = pd.concat([df3, df4], axis = 0)

df_final.tail(20)

Fecha,Tipo de Cambio
2021-06-01,19.9442
2021-07-01,19.8157
2021-08-01,19.8448
2021-09-01,20.1403
2021-10-01,20.4977
2021-11-01,20.3255
2021-12-01,21.7333
2022-01-01,20.5157
2021-01-31,20.2697
2021-02-28,20.8523


In [9]:
# Save the DataFrame as CSV
df_final.to_csv("/Users/portfedh/Desktop/FXBanxico_Reporte_Portafolio.csv", index=True)